In [1]:
%load_ext autoreload
%autoreload 2

from survshap import survlime
import pandas as pd
import numpy as np 
import pickle
from tqdm import tqdm
np.random.seed(123)

import numpy as np
import pandas as pd
from math import sqrt
import scipy as sp

import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error


from sksurv.util import Surv
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.ensemble import RandomSurvivalForest
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler


# Our very own survLime!
import survLime
from survLime.datasets.load_datasets import Loader 
from survLime import survlime_explainer


dataset = 'heart'
loader = Loader(dataset_name=dataset)
x,events, times = loader.load_data()



train, val, test = loader.preprocess_datasets(x, events, times, random_seed=0, standarize=True)
events_train = [x[0] for x in train[1]]
times_train  = [x[1] for x in train[1]]

events_val = [x[0] for x in val[1]]
times_val  = [x[1] for x in val[1]]

events_test = [x[0] for x in test[1]]
times_test  = [x[1] for x in test[1]]



model = CoxPHSurvivalAnalysis(alpha=0.0001)
#model = RandomSurvivalForest()
model.fit(train[0], train[1])

<frozen importlib._bootstrap>:219: RuntimeWarning: sklearn.tree._splitter.Splitter size changed, may indicate binary incompatibility. Expected 360 from C header, got 1160 from PyObject
/home/carlos.hernandez/dl/lib/python3.8/site-packages/pandas/core/series.py:4530: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


CoxPHSurvivalAnalysis(alpha=0.0001)

In [2]:
from survshap.survshap.explainer import SurvivalModelExplainer
cph_exp = SurvivalModelExplainer(model, train[0], train[1])

### kernel and neighbourhood based on SurvLIME paper
def kernel(distance):
    return 1 - np.sqrt(distance / 0.5)
def generate_neighbours(ind):
    count_neighbours = 0
    neighbours = np.zeros((1000, 11))
    while count_neighbours < 1000:
        neigh = np.random.random(11) - 0.5
        if np.sqrt(np.sum(neigh**2)) <= 0.5:
            neighbours[count_neighbours, ] = neigh
            count_neighbours += 1
    #import ipdb;ipdb.set_trace()
    neighbours += test[0].iloc[[ind]].values
    neighbours[0, ] = test[0].iloc[[ind]].values
    return neighbours



In [3]:
from survshap.survlime import SurvLIME
n_obs = len(test[0])
sls = [None for i in range(n_obs)]
funcs_dist = [None for i in range(n_obs)]
for i, obs in tqdm(enumerate(test[0].values)):
    xx = pd.DataFrame(np.atleast_2d(obs), columns=cph_exp.data.columns)
    survlime = SurvLIME(N=1000)
    survlime.fit(cph_exp, xx, kernel=kernel, neighbourhood=generate_neighbours(i), k=1)
    sls[i] = survlime
    break
    

0it [00:10, ?it/s]


In [5]:
%matplotlib inline
import plotly.io as pio
pio.renderers.default = "iframe"
survlime.plot(show=True)

'4.14.3'